### Import Libraries

In [1]:
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
from transformers import ConvNextModel, ConvNextImageProcessor
from config import CFG
from datasets import load_dataset
from PIL import Image

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
import time

import optuna
from optuna.samplers import TPESampler

from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

2024-01-15 09:45:41.641718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
label_data = pd.read_csv(os.path.join(CFG.train_data_dir, '文件标签汇总数据.csv'))
train_csv_folder = os.path.join(CFG.train_data_dir, 'csv文件')
test_A_csv_folder = os.path.join(CFG.test_A_data_dir, 'csv文件')
train_image_folder = os.path.join(CFG.project_name, 'project/image/训练集数据')
test_A_image_folder = os.path.join(CFG.project_name, 'project/image/A榜测试集数据')
label2id = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4}
id2label = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}

In [3]:
processor = ConvNextImageProcessor.from_pretrained(CFG.convNext_model)
model = ConvNextModel.from_pretrained(CFG.convNext_model)

In [4]:
X_train = []
y_train = []
for img in tqdm(os.listdir(train_image_folder)):
    img_path = os.path.join(train_image_folder, img)
    label = label_data[label_data['fileName'].str.contains(img[2:-4])]['defectType'].values[0]
    image = Image.open(img_path)
    # image 是单通道，需要转换成三通道
    image = np.array(image)
    image = np.stack([image, image, image], axis=-1)
    inputs = processor(image, return_tensors="pt")
    outputs = model(**inputs)
    X_train.append(outputs.pooler_output.squeeze().detach().numpy())
    y_train.append(label)
y_train = [label2id[i] for i in y_train]

100%|██████████| 115/115 [00:23<00:00,  4.81it/s]


In [5]:
X_test = []
name_test = []
test_name_list = os.listdir(test_A_csv_folder)
for img in tqdm(os.listdir(test_A_image_folder)):
    img_path = os.path.join(test_A_image_folder, img)
    image = Image.open(img_path)
    # image 是单通道，需要转换成三通道
    image = np.array(image)
    image = np.stack([image, image, image], axis=-1)
    inputs = processor(image, return_tensors="pt")
    outputs = model(**inputs)
    X_test.append(outputs.pooler_output.squeeze().detach().numpy())
    for name in test_name_list:
        if img[:-4] in name:
            name_test.append(name)
            break

100%|██████████| 115/115 [00:21<00:00,  5.44it/s]


### Optuna 

#### XGBoost

In [14]:
def objective(trial,data=X_train,target=y_train):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=2020,stratify=target)
    param = {
        'num_class': 5,
        'objective': 'multiclass',
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        # 'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        # 'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        # 'lambda': trial.suggest_float('lambda', 1e-3, 100),
        'lambda': 4.481503467107979,
        'alpha': trial.suggest_float('alpha', 1e-2, 0.5),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'colsample_bytree': 0.1748167867084523,
        # 'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        # 'subsample': trial.suggest_float('subsample', 0.1, 1),
        'subsample': 0.9509438953418313,
        # 'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'learning_rate': 0.8979650923282372,
        'n_estimators': 5000,
        # 'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }
    model = XGBClassifier(**param)
    
    model.fit(train_x,train_y,verbose=False)
    
    preds = model.predict(test_x)
    
    score = f1_score(test_y, preds, average=None)
    f1score = np.average(score, weights=[0.1, 0.3, 0.2, 0.1, 0.3])
    
    return f1score

In [15]:
storage_name = "sqlite:///optuna.db"
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize",
    study_name="IDD_XGB", storage=storage_name,load_if_exists=True
)
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_value = study.best_value
print("\n\nbest_value = "+str(best_value))
print("best_params:")
print(best_params)

[I 2024-01-15 14:08:18,268] Using an existing study with name 'IDD_XGB' instead of creating a new one.


[I 2024-01-15 14:08:37,329] Trial 338 finished with value: 0.5241258741258742 and parameters: {'alpha': 0.333226813926387, 'max_depth': 63, 'random_state': 2020, 'min_child_weight': 5}. Best is trial 250 with value: 0.8812687312687313.
[I 2024-01-15 14:08:55,108] Trial 339 finished with value: 0.5141258741258742 and parameters: {'alpha': 0.3007334222728416, 'max_depth': 62, 'random_state': 2020, 'min_child_weight': 5}. Best is trial 250 with value: 0.8812687312687313.
[I 2024-01-15 14:09:14,117] Trial 340 finished with value: 0.6154545454545455 and parameters: {'alpha': 0.3113333508099147, 'max_depth': 63, 'random_state': 2020, 'min_child_weight': 6}. Best is trial 250 with value: 0.8812687312687313.
[I 2024-01-15 14:09:31,892] Trial 341 finished with value: 0.5141258741258742 and parameters: {'alpha': 0.2905773157748701, 'max_depth': 59, 'random_state': 2020, 'min_child_weight': 5}. Best is trial 250 with value: 0.8812687312687313.
[I 2024-01-15 14:09:50,533] Trial 342 finished with v



best_value = 0.8812687312687313
best_params:
{'alpha': 0.025360289237833955, 'subsample': 0.9509438953418313, 'max_depth': 33, 'random_state': 2020, 'min_child_weight': 1}


In [22]:
p6 = {'num_class': 5,
      'objective': 'multiclass',
      'tree_method':'gpu_hist',
      'alpha': 0.025360289237833955, 
      'lambda': 4.481503467107979,
      'subsample': 0.9509438953418313, 
      'colsample_bytree': 0.1748167867084523,
      'learning_rate': 0.8979650923282372,
      'n_estimators': 5000,
      'max_depth': 33, 
      'random_state': 2020, 
      'min_child_weight': 1}

In [23]:
xgb = XGBClassifier(**p6)
xgb.fit(X_train, y_train)

XGBClassifier(alpha=0.025360289237833955, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.1748167867084523, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=4.481503467107979,
              learning_rate=0.8979650923282372, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=33, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5000, n_jobs=None, ...)

In [24]:
preds = xgb.predict(X_train)
score = f1_score(y_train, preds, average=None)
f1score = np.average(score, weights=[0.1, 0.3, 0.2, 0.1, 0.3])
print(f1score)

1.0


#### LGBMClassifier

In [25]:
# LGBMClassifier
def objective(trial,data=X_train,target=y_train):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,stratify=target,random_state=42)
    param = {
        'verbose': -1,
        'num_class': 5,
        'objective': 'multiclass',
        'random_state': 2020,
        'n_estimators': 10000,
        'n_iter': trial.suggest_int('n_iter', 1000, 10000),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'max_bin': trial.suggest_int('max_bin', 100, 10000),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-5, 100),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        # 'subsample': trial.suggest_float('subsample', 0.1, 0.9),
        'subsample': 0.6702911710631169,
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 120),
    }
    model = LGBMClassifier(**param)
    model.fit(train_x,train_y)
    
    preds = model.predict(test_x)
    
    score = f1_score(test_y, preds, average=None)
    f1score = np.average(score, weights=[0.1, 0.3, 0.2, 0.1, 0.3])
    
    return f1score

In [26]:
storage_name = "sqlite:///optuna.db"
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize",
    study_name="IDD_LGMB", storage=storage_name,load_if_exists=True
)
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_value = study.best_value
print("\n\nbest_value = "+str(best_value))
print("best_params:")
print(best_params)

[I 2024-01-15 14:34:47,255] Using an existing study with name 'IDD_LGMB' instead of creating a new one.


[I 2024-01-15 14:35:03,057] Trial 92 finished with value: 0.7858974358974359 and parameters: {'n_iter': 6122, 'max_depth': 25, 'max_bin': 4322, 'colsample_bytree': 0.7628685699079147, 'lambda_l1': 0.16836489759968754, 'lambda_l2': 55.49111774933534, 'learning_rate': 0.010027494118980011, 'min_data_in_leaf': 13}. Best is trial 84 with value: 0.806923076923077.
[I 2024-01-15 14:35:17,246] Trial 93 finished with value: 0.6095238095238096 and parameters: {'n_iter': 6270, 'max_depth': 24, 'max_bin': 4279, 'colsample_bytree': 0.7640259261444523, 'lambda_l1': 1.2378588089162403, 'lambda_l2': 55.82237610867699, 'learning_rate': 0.007004079936737532, 'min_data_in_leaf': 13}. Best is trial 84 with value: 0.806923076923077.
[I 2024-01-15 14:35:33,154] Trial 94 finished with value: 0.6095238095238096 and parameters: {'n_iter': 6165, 'max_depth': 24, 'max_bin': 4043, 'colsample_bytree': 0.7663367945348399, 'lambda_l1': 0.11654888163702476, 'lambda_l2': 56.59981500458606, 'learning_rate': 0.00496765



best_value = 0.8769230769230768
best_params:
{'n_iter': 6211, 'max_depth': 27, 'max_bin': 4031, 'colsample_bytree': 0.7748965702082932, 'lambda_l1': 0.003400734714471776, 'lambda_l2': 52.32055539449278, 'learning_rate': 0.055719599012254484, 'min_data_in_leaf': 11}


In [27]:
best_params['subsample'] = 0.6702911710631169
best_params['num_class'] = 5
best_params['objective'] = 'multiclass'
best_params['random_state'] = 2020
best_params['n_estimators'] = 10000
best_params['verbose'] = -1

In [28]:
lgbm = LGBMClassifier(**best_params)
lgbm.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.7748965702082932,
               lambda_l1=0.003400734714471776, lambda_l2=52.32055539449278,
               learning_rate=0.055719599012254484, max_bin=4031, max_depth=27,
               min_data_in_leaf=11, n_estimators=10000, n_iter=6211,
               num_class=5, objective='multiclass', random_state=2020,
               subsample=0.6702911710631169, verbose=-1)

#### CatBoostClassifier

In [34]:
CatBoostClassifier?

Init signature:
CatBoostClassifier(
    iterations=None,
    learning_rate=None,
    depth=None,
    l2_leaf_reg=None,
    model_size_reg=None,
    rsm=None,
    loss_function=None,
    border_count=None,
    feature_border_type=None,
    per_float_feature_quantization=None,
    input_borders=None,
    output_borders=None,
    fold_permutation_block=None,
    od_pval=None,
    od_wait=None,
    od_type=None,
    nan_mode=None,
    counter_calc_method=None,
    leaf_estimation_iterations=None,
    leaf_estimation_method=None,
    thread_count=None,
    random_seed=None,
    use_best_model=None,
    best_model_min_trees=None,
    verbose=None,
    silent=None,
    logging_level=None,
    metric_period=None,
    ctr_leaf_count_limit=None,
    store_all_simple_ctr=None,
    max_ctr_complexity=None,
    has_time=None,
    allow_const_label=None,
    target_border=None,
    classes_count=None,
    class_weights=None,
    auto_class_weights=None,
    class_names=None,
    one_hot_max_size=Non

In [45]:
# CatBoostClassifier
def objective(trial,data=X_train,target=y_train):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,stratify=target,random_state=42)
    param = {
        'objective': 'MultiClass',
        'random_state': 2020,
        'iterations': 1000,
        'depth': trial.suggest_int('depth', 1, 16),
        'max_bin': trial.suggest_int('max_bin', 100, 10000),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 120),
    }
    model = CatBoostClassifier(**param)
    model.fit(train_x,train_y)
    
    preds = model.predict(test_x)
    
    score = f1_score(test_y, preds, average=None)
    f1score = np.average(score, weights=[0.1, 0.3, 0.2, 0.1, 0.3])
    
    return f1score

In [46]:
storage_name = "sqlite:///optuna.db"
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize",
    study_name="IDD_CAT", storage=storage_name,load_if_exists=True
)
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_value = study.best_value
print("\n\nbest_value = "+str(best_value))
print("best_params:")
print(best_params)

[I 2024-01-15 15:38:44,451] Using an existing study with name 'IDD_CAT' instead of creating a new one.


0:	learn: 1.5678923	total: 1.02ms	remaining: 1.02s
1:	learn: 1.5437998	total: 2.57ms	remaining: 1.28s
2:	learn: 1.5294087	total: 3.46ms	remaining: 1.15s
3:	learn: 1.5205621	total: 4.38ms	remaining: 1.09s
4:	learn: 1.5149731	total: 5.28ms	remaining: 1.05s
5:	learn: 1.5113502	total: 6.19ms	remaining: 1.02s
6:	learn: 1.5089455	total: 7.17ms	remaining: 1.02s
7:	learn: 1.5073148	total: 8.09ms	remaining: 1s
8:	learn: 1.5061875	total: 8.95ms	remaining: 986ms
9:	learn: 1.4884973	total: 22.6ms	remaining: 2.24s
10:	learn: 1.4877121	total: 23.5ms	remaining: 2.11s
11:	learn: 1.4871522	total: 25.2ms	remaining: 2.07s
12:	learn: 1.4867478	total: 26.1ms	remaining: 1.98s
13:	learn: 1.4864524	total: 27.1ms	remaining: 1.91s
14:	learn: 1.4862345	total: 28ms	remaining: 1.83s
15:	learn: 1.4860726	total: 28.8ms	remaining: 1.77s
16:	learn: 1.4859514	total: 29.7ms	remaining: 1.72s
17:	learn: 1.4858602	total: 30.6ms	remaining: 1.67s
18:	learn: 1.4857913	total: 31.5ms	remaining: 1.62s
19:	learn: 1.4857389	total:

[I 2024-01-15 15:38:45,833] Trial 13 finished with value: 0.4073529411764706 and parameters: {'depth': 7, 'max_bin': 4177, 'colsample_bylevel': 0.2106062780128784, 'l2_leaf_reg': 40.03308107961289, 'learning_rate': 0.8044006001065191, 'bagging_temperature': 80.11338618400185, 'min_data_in_leaf': 44}. Best is trial 3 with value: 0.6369230769230769.


868:	learn: 1.2549917	total: 948ms	remaining: 143ms
869:	learn: 1.2549917	total: 949ms	remaining: 142ms
870:	learn: 1.2549917	total: 950ms	remaining: 141ms
871:	learn: 1.2549917	total: 951ms	remaining: 140ms
872:	learn: 1.2549917	total: 952ms	remaining: 138ms
873:	learn: 1.2549917	total: 953ms	remaining: 137ms
874:	learn: 1.2549917	total: 954ms	remaining: 136ms
875:	learn: 1.2549917	total: 954ms	remaining: 135ms
876:	learn: 1.2549917	total: 955ms	remaining: 134ms
877:	learn: 1.2549917	total: 956ms	remaining: 133ms
878:	learn: 1.2549917	total: 957ms	remaining: 132ms
879:	learn: 1.2549917	total: 958ms	remaining: 131ms
880:	learn: 1.2549917	total: 958ms	remaining: 129ms
881:	learn: 1.2549917	total: 959ms	remaining: 128ms
882:	learn: 1.2549917	total: 960ms	remaining: 127ms
883:	learn: 1.2549917	total: 961ms	remaining: 126ms
884:	learn: 1.2549917	total: 962ms	remaining: 125ms
885:	learn: 1.2549917	total: 962ms	remaining: 124ms
886:	learn: 1.2549917	total: 963ms	remaining: 123ms
887:	learn: 

[I 2024-01-15 15:38:56,922] Trial 14 finished with value: 0.4492647058823529 and parameters: {'depth': 16, 'max_bin': 5105, 'colsample_bylevel': 0.8280374745769203, 'l2_leaf_reg': 29.879963685561147, 'learning_rate': 0.1918154861752795, 'bagging_temperature': 65.94377806397554, 'min_data_in_leaf': 55}. Best is trial 3 with value: 0.6369230769230769.


...

998:	learn: 0.1260573	total: 3m 27s	remaining: 208ms
999:	learn: 0.1259026	total: 3m 27s	remaining: 0us
0:	learn: 1.5733541	total: 8.38ms	remaining: 8.38s
1:	learn: 1.5537610	total: 13.8ms	remaining: 6.87s
2:	learn: 1.5263393	total: 18.4ms	remaining: 6.1s
3:	learn: 1.5018608	total: 22.8ms	remaining: 5.67s
4:	learn: 1.4876176	total: 27.1ms	remaining: 5.39s
5:	learn: 1.4766882	total: 31.6ms	remaining: 5.24s
6:	learn: 1.4641028	total: 36.7ms	remaining: 5.2s
7:	learn: 1.4509312	total: 40.7ms	remaining: 5.05s
8:	learn: 1.4402678	total: 53.8ms	remaining: 5.92s
9:	learn: 1.4315069	total: 57.9ms	remaining: 5.73s
10:	learn: 1.4195819	total: 61.4ms	remaining: 5.52s
11:	learn: 1.3953459	total: 65.1ms	remaining: 5.36s
12:	learn: 1.3769775	total: 68.8ms	remaining: 5.22s
13:	learn: 1.3636264	total: 72.8ms	remaining: 5.13s
14:	learn: 1.3368064	total: 76.7ms	remaining: 5.04s
15:	learn: 1.3178877	total: 80.4ms	remaining: 4.95s
16:	learn: 1.3137618	total: 92.9ms	remaining: 5.37s
17:	learn: 1.3046790	tot

[I 2024-01-15 15:54:02,259] Trial 40 finished with value: 0.8502564102564103 and parameters: {'depth': 1, 'max_bin': 6689, 'colsample_bylevel': 0.8997817091061128, 'l2_leaf_reg': 84.39456253724344, 'learning_rate': 0.7418796107974877, 'bagging_temperature': 15.267120114275524, 'min_data_in_leaf': 29}. Best is trial 19 with value: 0.8551515151515152.


992:	learn: 0.1355474	total: 5.2s	remaining: 36.6ms
993:	learn: 0.1354909	total: 5.2s	remaining: 31.4ms
994:	learn: 0.1354376	total: 5.21s	remaining: 26.2ms
995:	learn: 0.1353722	total: 5.21s	remaining: 20.9ms
996:	learn: 0.1352314	total: 5.21s	remaining: 15.7ms
997:	learn: 0.1351295	total: 5.22s	remaining: 10.5ms
998:	learn: 0.1348368	total: 5.22s	remaining: 5.23ms
999:	learn: 0.1347025	total: 5.23s	remaining: 0us
0:	learn: 1.6069798	total: 550ms	remaining: 9m 9s
1:	learn: 1.6044903	total: 27.8s	remaining: 3h 51m 10s
2:	learn: 1.6012650	total: 27.8s	remaining: 2h 34m 9s
3:	learn: 1.5990377	total: 55.6s	remaining: 3h 50m 49s
4:	learn: 1.5963521	total: 55.9s	remaining: 3h 5m 16s
5:	learn: 1.5942620	total: 1m	remaining: 2h 48m 4s
6:	learn: 1.5920407	total: 1m 3s	remaining: 2h 29m 41s
7:	learn: 1.5888602	total: 1m 5s	remaining: 2h 15m 53s
8:	learn: 1.5861248	total: 1m 35s	remaining: 2h 55m 36s
9:	learn: 1.5839967	total: 2m 2s	remaining: 3h 22m 46s
10:	learn: 1.5809971	total: 2m 3s	remaini

[W 2024-01-15 16:11:14,020] Trial 41 failed with parameters: {'depth': 14, 'max_bin': 3955, 'colsample_bylevel': 0.7958825823138771, 'l2_leaf_reg': 90.96826989859488, 'learning_rate': 0.21659765478775067, 'bagging_temperature': 4.931158089263598, 'min_data_in_leaf': 43} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home/dxd/.local/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_36564/20216974.py", line 17, in objective
    model.fit(train_x,train_y)
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/catboost/core.py", line 5100, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/catboost/core.py", line 2319, in _fit
    self._train(
  File "/ai/anaconda3/envs/kaggle/lib/python3.8/

KeyboardInterrupt: 

In [50]:
best_params_cat = study.best_params
best_params_cat['iterations'] = 1000
best_params_cat['objective'] = 'MultiClass'
best_params_cat['random_state'] = 2020

In [51]:
cat = CatBoostClassifier(**best_params_cat)
cat.fit(X_train, y_train)

0:	learn: 1.5720533	total: 22.6ms	remaining: 22.6s
1:	learn: 1.5547576	total: 63.6ms	remaining: 31.7s
2:	learn: 1.5328609	total: 104ms	remaining: 34.6s
3:	learn: 1.5059918	total: 145ms	remaining: 36.1s
4:	learn: 1.4923310	total: 186ms	remaining: 37.1s
5:	learn: 1.4732942	total: 228ms	remaining: 37.7s
6:	learn: 1.4459405	total: 269ms	remaining: 38.1s
7:	learn: 1.4296678	total: 309ms	remaining: 38.4s
8:	learn: 1.4109412	total: 351ms	remaining: 38.6s
9:	learn: 1.3898350	total: 393ms	remaining: 38.9s
10:	learn: 1.3482412	total: 434ms	remaining: 39s
11:	learn: 1.3137875	total: 476ms	remaining: 39.2s
12:	learn: 1.2872299	total: 517ms	remaining: 39.2s
13:	learn: 1.2639968	total: 559ms	remaining: 39.4s
14:	learn: 1.2179132	total: 601ms	remaining: 39.5s
15:	learn: 1.1988862	total: 643ms	remaining: 39.5s
16:	learn: 1.1744069	total: 685ms	remaining: 39.6s
17:	learn: 1.1595176	total: 727ms	remaining: 39.6s
18:	learn: 1.1388267	total: 769ms	remaining: 39.7s
19:	learn: 1.1264895	total: 810ms	remaini

### Predict

In [ ]:
pred = xgb.predict(X_test)
pred = [id2label[i] for i in pred]
pred = pd.DataFrame({'fileName':name_test, 'defectType':pred})
pred.to_csv('xgb.csv', index=False)

In [7]:
clf = GaussianNB()
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6={'n_iter': 5000,
    'verbose': -1,
    'objective': 'multiclass',
    'num_class': 5,
    'learning_rate': 0.01, 
    'colsample_bytree': 0.78,
    'colsample_bynode': 0.8, 
    'lambda_l1': 5, 
    'lambda_l2': 3, 
    'min_data_in_leaf': 115, 
    'max_depth': 30, 
    'max_bin': 1000}
lgb=LGBMClassifier(**p6)
cat=CatBoostClassifier(iterations=3000,
                       verbose=0,
                       l2_leaf_reg=6.7,
                       learning_rate=0.005,
                       subsample = 0.4,
                       bootstrap_type='Bernoulli',
                       allow_const_label=True,
                       loss_function = 'MultiClass')

weights = [0.25,0.25,0.25,0.25]
print('Training...')
now = time.time()
ensemble = VotingClassifier(estimators=[('mnb',clf),
                                        ('sgd', sgd_model),
                                        ('lgb',lgb), 
                                        ('cat', cat)
                                        ],
                            weights=weights, 
                            voting='soft',
                            n_jobs=-1)
ensemble.fit(X_train, y_train)
print('Training time: ', time.time() - now)
print('Training finished')

Training...


/ai/anaconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training time:  179.50280809402466
Training finished


In [8]:
print('Predicting...')
now = time.time()
y_pred = ensemble.predict(X_test)
print('Predicting time: ', time.time() - now)
print('Predicting finished')

Predicting...
Predicting time:  0.2310347557067871
Predicting finished


In [9]:
sub = pd.DataFrame(columns=['defectType', 'fileName'])
sub['fileName'] = name_test
sub['defectType'] = [id2label[i] for i in y_pred]
sub.to_csv('submission.csv', index=False)